In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-01-21


Found rundate 2024-01-20 10:00:02 < 2024-01-21 at commit 2cf77f2b; returning commit 0bd2a05


'2024-01-21 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-01-21 10:00:02, NJSP was reporting 26 YTD deaths
Current YTD Deaths (2025-01-21 10:00:01-05:00): 22
Previous year YTD Deaths (adjusted; 2024-01-21 10:00:02): 26.07121809711861
Projected 2025 total: 683.1032615232847
-15.6% change


FAUQStats cache miss: 2024, Sun Jan 21 10:00:02 EST 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,NaN,State Highway 1,1.0,0.0,0.0,1.0,0.0,1,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,NaN,State Highway 129,1.0,1.0,0.0,0.0,0.0,129,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,NaN,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,446,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,East Oak St,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,NaN,State Highway 27,1.0,0.0,0.0,1.0,0.0,27,NaN,2024-01-06 08:37:00-05:00
12953,08,Gloucester,0802,Deptford Twsp,NaN,State Highway 47,1.0,1.0,0.0,0.0,0.0,47,NaN,2024-01-06 12:10:00-05:00
12928,15,Ocean,1505,Berkeley Twsp,NaN,State Highway 9 (Business),1.0,0.0,0.0,1.0,0.0,9,NaN,2024-01-06 18:44:00-05:00
12930,13,Monmouth,1334,Neptune Twsp,NaN,State Highway 33,1.0,0.0,1.0,0.0,0.0,33,NaN,2024-01-07 15:55:00-05:00
12929,12,Middlesex,1205,Edison Twsp,NaN,State Highway 1,1.0,0.0,0.0,1.0,0.0,1,NaN,2024-01-08 02:20:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,1,0,1,0,2
Bergen,2,0,0,0,2
Burlington,0,0,1,0,1
Camden,4,0,1,0,5
Cape May,1,0,0,0,1
Mercer,0,0,1,0,1
Middlesex,2,1,0,0,3
Ocean,1,1,1,0,3
Salem,1,0,0,0,1


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

5.6% through the year, 94.4% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,2,31,1
Bergen,driver,1,15,2
Burlington,driver,1,32,0
Camden,driver,0,19,4
Essex,driver,0,18,0
...,...,...,...,...
Morris,crashes,0,22,0
Salem,crashes,0,14,1
Somerset,crashes,0,19,0


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,53,4,29,2,19
Bergen,40,2,17,5,18
Burlington,44,1,29,3,10
Camden,50,0,26,4,15
Cape May,10,0,9,0,1
Cumberland,25,0,18,6,7
Essex,46,0,17,9,25
Gloucester,26,0,16,8,6
Hudson,16,1,8,2,7


In [9]:
projected.to_csv(PROJECTED_CSV)